In [ ]:
require 'ynab'
require 'active_support'
require 'active_support/core_ext'
require 'active_support/core_ext/numeric'

access_token = ENV['YNAB_TOKEN']

ynab_api = YNAB::API.new(access_token)

budget_id = ENNV['YNAB_BUDGET_ID']

categories_resp = ynab_api.categories.get_categories(budget_id)
category_groups = categories_resp.data.category_groups

categories = category_groups.flat_map { |group| group.categories }
                            .select { |cat| cat.goal_type == 'MF'}

transactions = categories.flat_map do |cat|
  ynab_api.transactions.get_transactions_by_category(budget_id, cat.id, since_date: Time.now - 4.weeks)
                       .data.transactions.map { |t| { category: cat, transaction: t}}
end

transactions.sort_by { |group| group[:transaction].date}
            .reverse
            .each do |group|
  c = group[:category]
  t = group[:transaction]
  puts "#{t.date} - #{c.name}, #{t.payee_name} - $#{t.amount/1000.0}"
end

nil
